In [ ]:
%pip install -r requirements.txt

# Imports

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import torch
from torch.utils.data import TensorDataset, DataLoader
from tsai.all import *
from sklearn.metrics import f1_score
from fastai.metrics import AccumMetric
import optuna
from optuna.integration import FastAIPruningCallback
from datetime import timedelta
import statistics

In [ ]:
my_setup(optuna)

In [ ]:
set_seed(42)

def set_every_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed = 42
set_every_seed(seed)
tqdm.pandas()

In [ ]:
pd.set_option("display.max_columns", 70)
pd.set_option("display.max_rows", 1000)

# Preparation

## Prepare dfs

In [ ]:
match_h1 = pd.read_csv(r"handball_sample\MD15_Flensburg_Melsungen\SG_Flensburg-Handewitt_vs._MT__phase_1.HZ_positions.csv", sep=";")
match_h2 = pd.read_csv(r"handball_sample\MD15_Flensburg_Melsungen\SG_Flensburg-Handewitt_vs._MT__phase_2.HZ_positions.csv", sep=";")
tags = pd.read_csv(r"handball_sample\MD15_Flensburg_Melsungen\tags_flensburg_melsungen_md15_s2324.csv", sep=";")

In [ ]:
tags

In [ ]:
match_h1.head()

In [ ]:
match_h2.tail()

In [ ]:
match_full = pd.concat([match_h1, match_h2]).reset_index().drop(["index", "heart rate in bpm", "core temperature in celsius", "player orientation in deg",  "Unnamed: 23"], axis=1)
match_full

In [ ]:
match_full["formatted local time"] = pd.to_datetime(match_full["formatted local time"])

In [ ]:
tags.drop(["player"], axis=1, inplace=True)

# Data Understanding

## General

In [ ]:
# See where multiple players get possession at same time
match_ball_poss = match_full[~np.isnan(match_full["ball possession (id of possessed ball)"])]
dups = match_ball_poss[match_ball_poss.duplicated(["ts in ms", "ball possession (id of possessed ball)"], keep=False)]
counts = dups.groupby(["ts in ms", "ball possession (id of possessed ball)"]).size()
rows_with_two_or_more = counts[counts >= 2].reset_index()
result = match_ball_poss.merge(rows_with_two_or_more, on=["ts in ms", "ball possession (id of possessed ball)"])
result

In [ ]:
grouped = match_full.groupby("formatted local time")["ball possession (id of possessed ball)"].count()
player_count_frequency = grouped.value_counts()
# Sort index for better readability
player_count_frequency = player_count_frequency.sort_index()

player_count_frequency = player_count_frequency.reset_index()
player_count_frequency.columns = ["number_of_players_in_possession", "frequency"]

In [ ]:
player_count_frequency

In [ ]:
print(match_full.info())

In [ ]:
match_full.describe()

# Checking if timestamps in video and tabular data are comparable

## Find unique players of both teams in the game to create the keys txt file

In [ ]:
keys_dict = {"league_id": [], "team": [], "jersey_number": [], "name": []}
for id in match_full["league id"].unique():
    if "Ball" not in id:
        row = match_full[match_full["league id"] == id].iloc[0]
        keys_dict["league_id"].append(row["league id"])
        keys_dict["team"].append(row["group name"])
        keys_dict["jersey_number"].append(row["number"])
        keys_dict["name"].append(row["full name"])

keys_df = pd.DataFrame(keys_dict)

In [ ]:
tags["tag text"].replace("20_kevin_moller", "20_kevin_møller", inplace=True)
tags["tag text"].replace("63_lasse_moller", "63_lasse_møller", inplace=True)
tags["tag text"].replace("11_petter_overby", "11_petter_øverby", inplace=True)
tags["tag text"].replace("71_elias_eliefsen_á_skipagotu", "71_elias_eliefsen_á_skipagøtu", inplace=True)

In [ ]:
tags["tag text"].unique()

In [ ]:
def change_tag_name(tag):
    if tag != "no_possession" and tag != "time_check_position" and tag != "game_start" and tag != "ball_not_visible":
        first_underscore_pos = tag.find("_")
        name = tag[first_underscore_pos + 1:]
        print(name)

for tag in tags["tag text"].unique():
    change_tag_name(tag)

In [ ]:
keys_df.sort_values(["team", "jersey_number"], ascending=[True, True])

## Check time

In [ ]:
match_full[~pd.isnull(match_full["player orientation category (forward, back, left, right)"])]

In [ ]:
match_start = match_full[match_full[~np.isnan(match_full["ball possession (id of possessed ball)"])].index[0]:]

In [ ]:
match_start_poss = match_start.dropna(subset=["ball possession (id of possessed ball)"])

# Drop consecutive rows where same player is still in possession
match_start_poss = match_start_poss[match_start_poss["sensor id"] != match_start_poss["sensor id"].shift(1)]

In [ ]:
match_start.loc[:, "time diff from start"] = match_start.loc[:, "formatted local time"] - match_start["formatted local time"].iloc[0]

### Convert tags timestamp to actual time

In [ ]:
tags[tags["tag text"].isin(["time_check_position"])]

In [ ]:
def round_to_nearest_50ms(dt):
    """ Round the timestamps to nearest 50ms.
    
    Parameters:
    dt -- the datetime timestamps

    Returns:
    rounded_dt -- the timestamps rounded to nearest 50ms
    """
    # Convert microseconds to milliseconds and round to nearest 50 milliseconds
    milliseconds = round(dt.microsecond / 1000.0 / 50.0) * 50
    # Rebuild datetime with new milliseconds, adjusting for overflow
    new_second = dt.second
    if milliseconds >= 1000:
        milliseconds -= 1000
        new_second += 1
    # Ensure seconds don't overflow
    if new_second >= 60:
        new_second -= 60
        dt += timedelta(minutes=1)
    rounded_dt = dt.replace(second=new_second, microsecond=int(milliseconds * 1000))
    return rounded_dt

In [ ]:
offset = 0
game = ""
tags["# time (in ms) vid"] = tags["# time (in ms)"]
tags["# time (in ms)"] = tags["# time (in ms)"] - tags["# time (in ms)"][tags["tag text"] == "game_start"].iloc[0]

# Get game start time
if game == "FLEvsEIS":
    table_start_time = pd.to_datetime("2023-10-28 19:02:14.000") # No player has possession in first half of that game according to Kinexon
# elif game == "FLEvsRNL":
#     table_start_time = pd.to_datetime("2023-11-18 18:02:10.650") # Possession starts to late
else:
    table_start_time = match_start_poss["formatted local time"].iloc[0]

 # Create adjusted timestamp column where video time of tags is converted to actual timestamp 
    tags["adjusted_timestamp"] = tags["# time (in ms)"].apply(lambda x: table_start_time + timedelta(milliseconds=x + offset))
    tags["rounded_timestamp"] = tags["adjusted_timestamp"].apply(round_to_nearest_50ms) # Round adjusted timestamps

In [ ]:
table_start_time

In [ ]:
# Forward fill tags df to have correct merging of match and tags df 
video_start_time = tags["rounded_timestamp"].iloc[0]
table_end_time = match_full["formatted local time"].iloc[-1]

all_timestamps = pd.date_range(start=video_start_time, end=table_end_time, freq="50L")
all_timestamps_df = pd.DataFrame(all_timestamps, columns=["timestamp"])

# Merge tags with all timestamps and forward fill
tags_full = pd.merge(all_timestamps_df, tags, left_on="timestamp", right_on="rounded_timestamp", how="left")
tags_full["tag text"].replace(["game_start", "no_possession", "time_check_position"], np.nan, inplace=True)
cols_from_tags = tags.columns.difference(["# time (in ms)", "adjusted_timestamp"])
for col in cols_from_tags:
    tags_full[col] = tags_full[col].ffill()

In [ ]:
merged_df = pd.merge(match_full, tags_full, left_on="formatted local time", right_on="timestamp", how="left")
merged_df

In [ ]:
tags[tags["tag text"].isin(["time_check_position"])]

In [ ]:
tag_to_check = 0
ts_to_check = tags[tags["tag text"].isin(["time_check_position"])]["rounded_timestamp"].iloc[tag_to_check]
# Find index where first of i-th check-tag is used
try:
    # Get previous and next timestamps based on timestamp to check for consideration
    check_min, check_max = match_full[match_full["formatted local time"] == ts_to_check].index[0] - 150, match_full[match_full["formatted local time"] == ts_to_check].index[0] + 165
except IndexError:
    # If timestamp to check is not available in data, proceed to next timestamp
    print(ts_to_check, "Data not available")

df_to_check = match_full[match_full["group name"] == "Ball"].loc[check_min:check_max]
if df_to_check.size == 0:
    print("Ball is not found in area to check")

if df_to_check["z in m"].min() > 5:
    # If z value is over 5m in every considered timestamp, proceed to next timestamp
    print(ts_to_check, "Z sensor in ball wrong, this position can't be reliably checked")

time_in_df = df_to_check["formatted local time"][df_to_check["z in m"] == df_to_check["z in m"].min()]

# Print difference between tabular and video times
difference = abs(time_in_df - ts_to_check)
print("Checkpoint", tag_to_check, ", Video", ts_to_check, ", Difference in ms is:", difference.iloc[0].microseconds/1000, ", Tabular", time_in_df.iloc[0])

In [ ]:
df_to_check

# Wrong data

In [ ]:
match_training_stats = pd.read_csv(r"handball_sample\match_training_stats.csv", index_col=0, sep=";")
match_test_stats = pd.read_csv(r"handball_sample\match_test_stats.csv", index_col=0, sep=";")
match_training_model = pd.read_csv(r"handball_sample\match_training_model.csv", index_col=0, sep=";")
match_test_model = pd.read_csv(r"handball_sample\match_test_model.csv", index_col=0, sep=";")

In [ ]:
match_training_stats[:15]

In [ ]:
grouped_train = match_training_model.groupby(by=["game"]).size()
grouped_test = match_test_model.groupby(by=["game"]).size()

In [ ]:
filtered_df_train = match_training_model[~match_training_model["group name"].str.contains("Ball")]
filtered_df_train = filtered_df_train[~filtered_df_train["tag text"].str.contains("not_visible")]
grouped_train = filtered_df_train.groupby("game").size()
num_rows_train = grouped_train.sum()
print("Number of rows after group by without 'Ball':", grouped_train, num_rows_train)

print("------")
filtered_df_test = match_test_model[~match_test_model["group name"].str.contains("Ball")]
grouped_test = filtered_df_test.groupby("game").size()
num_rows_test = grouped_test.sum()
print("Number of rows after group by without 'Ball':", grouped_test, num_rows_test)

In [ ]:
filtered_df_train = match_training_model[~match_training_model["group name"].str.contains("Ball")]
filtered_df_train = filtered_df_train[~filtered_df_train["tag text"].str.contains("not_visible")]
grouped_train = filtered_df_train.groupby("game").size()
num_rows_train = grouped_train.sum()
print("Number of rows after group by without 'Ball':", grouped_train, num_rows_train)

print("------")
filtered_df_test = match_test_model[~match_test_model["group name"].str.contains("Ball")]
filtered_df_test = filtered_df_test[~filtered_df_test["tag text"].str.contains("not_visible")]
grouped_test = filtered_df_test.groupby("game").size()
num_rows_test = grouped_test.sum()
print("Number of rows after group by without 'Ball':", grouped_test, num_rows_test)

In [ ]:
match_stats = pd.concat([match_training_stats, match_test_stats])
match_model = pd.concat([match_training_model, match_test_model])

In [ ]:
group_counts_train_stats = match_training_stats.groupby(by=["formatted local time", "game"])["group name"].agg(ball_count=lambda x: (x == "Ball").sum(),
                                                                                                               player_count=lambda x: (x != "Ball").sum()).reset_index()
group_counts_test_stats = match_test_stats.groupby(by=["formatted local time", "game"])["group name"].agg(ball_count=lambda x: (x == "Ball").sum(),
                                                                                                          player_count=lambda x: (x != "Ball").sum()).reset_index()
group_counts_train_model = match_training_model.groupby(by=["formatted local time", "game"])["group name"].agg(ball_count=lambda x: (x == "Ball").sum(),
                                                                                                               player_count=lambda x: (x != "Ball").sum()).reset_index()
group_counts_test_model = match_test_model.groupby(by=["formatted local time", "game"])["group name"].agg(ball_count=lambda x: (x == "Ball").sum(),
                                                                                                          player_count=lambda x: (x != "Ball").sum()).reset_index()

In [ ]:
group_counts_stats = pd.concat([group_counts_train_stats, group_counts_test_stats])
group_counts_model = pd.concat([group_counts_train_model, group_counts_test_model])

In [ ]:
# Usable timestamps 
ts_counts = match_full["formatted local time"].value_counts()
usable_ts = ts_counts[ts_counts >= 15].index
match_full = match_full[match_full["formatted local time"].isin(usable_ts)]
ts_counts_over_15 = match_full["formatted local time"].value_counts()
usable_ts_over_15 = ts_counts[ts_counts > 15].index
match_over_15 = match_full[match_full["formatted local time"].isin(usable_ts_over_15)]
match_over_15["formatted local time"].value_counts()

In [ ]:
timestamps_with_no_possession = match_full.groupby("formatted local time").filter(lambda x: x["ball possession (id of possessed ball)"].isna().all())

# Unique timestamps where no player has possession of ball
unique_timestamps = timestamps_with_no_possession["formatted local time"].nunique()
unique_timestamps_total = match_full["formatted local time"].nunique()
unique_timestamps_without_inv = merged_df[merged_df["tag text"] != "ball_not_visible"]["formatted local time"].nunique()

print(unique_timestamps) 
print(unique_timestamps_total)
print(unique_timestamps_without_inv)

In [ ]:
# Timestamps with wrong z-column sensor data
match_full[match_full["z in m"] > 10]["formatted local time"].unique()

# Statistics

## Statistics for data in general

In [ ]:
for group_counts in [group_counts_stats, group_counts_model]:
    ball_game_counts = group_counts.groupby(by=["game"], sort=False)["ball_count"].value_counts(sort=False)
    ball_game_counts = ball_game_counts.reset_index()
    ball_game_counts.columns = ["number_of_balls_in_game", "frequency", "game"]

    sns.set_theme(style="whitegrid")
    ax = sns.catplot(x="number_of_balls_in_game", y="game", hue="frequency", kind="bar", data=ball_game_counts, palette="ch:s=.25,rot=-.25", height=6)

    ax._legend.set_bbox_to_anchor((1, 0.9))

    plt.title("Frequency of balls in timestamps each game", fontsize=20)
    plt.xlabel("Game and number of balls", fontsize=15)
    plt.ylabel("Frequency", fontsize=15)

    for ax in ax.axes.flat:
        for p in ax.patches:
            if p.get_width() != 0 and p.get_height() != 0:
                ax.text(p.get_x() + p.get_width() / 2., p.get_height(), f"{int(p.get_height())}",
                        fontsize=11, ha="center", va="bottom")

    plt.xticks()
    plt.tight_layout()
    plt.show()

In [ ]:
for group_counts in [group_counts_stats, group_counts_model]:
    player_game_counts = group_counts.groupby(by=["game"], sort=False)["player_count"].value_counts(sort=False)
    player_game_counts_df = pd.DataFrame(player_game_counts).reset_index()
    player_game_counts_df.columns = ["game", "number_of_players_in_timestamp", "frequency"]
    for game in player_game_counts_df["game"].unique():
        print(game)
        game_df = player_game_counts_df[player_game_counts_df["game"] == game]

        ax = sns.barplot(x="number_of_players_in_timestamp", y="frequency", data=game_df, palette="ch:s=.25,rot=-.25")

        plt.title(f"Frequency of players in a timestamp in game {game}", fontsize=14)
        plt.xlabel("Number of players in timestamp", fontsize=12)
        plt.ylabel("Frequency", fontsize=12)

        for p in ax.patches:
            ax.text(p.get_x() + p.get_width()/2., p.get_height(), f"{int(p.get_height())}", 
                    fontsize=12, ha="center", va="bottom")
        
        sns.set_theme(style="whitegrid")

        plt.xticks()
        plt.tight_layout()
        plt.show()

In [ ]:
grouped_test = match_stats.groupby(by=["formatted local time", "game"])["ball possession (id of possessed ball)"].count()
player_count_frequency_test = grouped_test.groupby(by=["game"], sort=False).value_counts(sort=False)
player_count_frequency_test = player_count_frequency_test.reset_index()
player_count_frequency_test.columns = ["game", "number_of_players_in_possession", "frequency"]
player_count_frequency_test

In [ ]:
for match in [match_stats, match_model]:
    grouped = match.groupby(by=["formatted local time", "game"])["ball possession (id of possessed ball)"].count()
    player_count_frequency = grouped.groupby(by=["game"], sort=False).value_counts(sort=False)
    player_count_frequency = player_count_frequency.reset_index()
    player_count_frequency.columns = ["game", "number_of_players_in_possession", "frequency"]

    for game in player_count_frequency["game"].unique():
        print(game)
        game_df = player_count_frequency[player_count_frequency["game"] == game]

        sns.set_theme(style="whitegrid")
        ax = sns.barplot(x="number_of_players_in_possession", y="frequency", data=game_df, palette="ch:s=.25,rot=-.25")

        plt.title(f"Frequency of number of players in possession in game {game}", fontsize=14)
        plt.xlabel("Number of players in possession", fontsize=12)
        plt.ylabel("Frequency", fontsize=12)

        for p in ax.patches:
            ax.text(p.get_x() + p.get_width()/2., p.get_height(), f"{int(p.get_height())}", 
                    fontsize=12, ha="center", va="bottom")
        

        plt.xticks()
        plt.tight_layout()
        plt.show()

In [ ]:
filtered_merged_df_train_stats = pd.read_csv(r"handball_sample\match_training_stats.csv", sep=";", index_col=0)
filtered_merged_df_test_stats = pd.read_csv(r"handball_sample\match_test_stats.csv", sep=";", index_col=0)
filtered_merged_df_train_model = pd.read_csv(r"handball_sample\match_training_model.csv", sep=";", index_col=0)
filtered_merged_df_test_model = pd.read_csv(r"handball_sample\match_test_model.csv", sep=";", index_col=0)

In [ ]:
filtered_merged_df_stats = pd.concat([filtered_merged_df_train_stats, filtered_merged_df_test_stats])
filtered_merged_df_model = pd.concat([filtered_merged_df_train_model, filtered_merged_df_test_model])

In [ ]:
filtered_merged_df_stats.columns

In [ ]:
# Show wrong data for all timestamps, including ones where no one in that timestamp has possession
def select_row(group):
    # Check if any row has non-NaN "ball possession" value
    possession_rows = group.dropna(subset=["ball possession (id of possessed ball)"])
    
    if not possession_rows.empty:
        # Return row with ball possession
        return possession_rows
    else:
        # Return first row of group
        first_row = group.iloc[[0]].copy()
        first_row["full name"] += "no_possession"
        return first_row

In [ ]:
selected_rows_df_stats = filtered_merged_df_stats.groupby(by=["formatted local time"]).progress_apply(select_row).reset_index(drop=True)
selected_rows_df_stats["tag_text_check"] = selected_rows_df_stats["full name"] == selected_rows_df_stats["tag text"]
individual_game_stats = selected_rows_df_stats.groupby("game")["tag_text_check"].value_counts()
print(individual_game_stats)

In [ ]:
selected_rows_df_model = filtered_merged_df_model.groupby(by=["formatted local time"]).progress_apply(select_row).reset_index(drop=True)
selected_rows_df_model["tag_text_check"] = selected_rows_df_model["full name"] == selected_rows_df_model["tag text"]
individual_game_model = selected_rows_df_model.groupby("game")["tag_text_check"].value_counts()
print(individual_game_model)

In [ ]:
poss_set = set(selected_rows_df_model[selected_rows_df_model["tag_text_check"] == True]["formatted local time"].unique())

In [ ]:
# Show wrong data only for timestamps where someone has ball according to Kinexon data 
filtered_with_no_poss_stats = filtered_merged_df_stats.groupby(by=["formatted local time", "game"]).filter(lambda x: not x["ball possession (id of possessed ball)"].isna().all())
filtered_no_poss_check_stats = filtered_with_no_poss_stats.dropna(subset=["ball possession (id of possessed ball)"])
filtered_no_poss_check_stats_copy = filtered_no_poss_check_stats.copy()
filtered_no_poss_check_stats_copy["tag_text_check"] = filtered_no_poss_check_stats_copy["tag text"] == filtered_no_poss_check_stats_copy["full name"]
individual_game_stats_other_stats = filtered_no_poss_check_stats_copy.groupby("game")["tag_text_check"].value_counts()
print(individual_game_stats_other_stats)

In [ ]:
# Show wrong data only for timestamps where someone has ball according to Kinexon data 
filtered_with_no_poss_model = filtered_merged_df_model.groupby(by=["formatted local time", "game"]).filter(lambda x: not x["ball possession (id of possessed ball)"].isna().all())
filtered_no_poss_check_model = filtered_with_no_poss_model.dropna(subset=["ball possession (id of possessed ball)"])
filtered_no_poss_check_model_copy = filtered_no_poss_check_model.copy()
filtered_no_poss_check_model_copy["tag_text_check"] = filtered_no_poss_check_model_copy["tag text"] == filtered_no_poss_check_model_copy["full name"]
individual_game_stats_other_model = filtered_no_poss_check_model_copy.groupby("game")["tag_text_check"].value_counts()
print(individual_game_stats_other_model)

In [ ]:
no_poss_set = set(filtered_no_poss_check_model_copy[filtered_no_poss_check_model_copy["tag_text_check"] == True]["formatted local time"].unique())
no_poss_set.difference(poss_set)

## Statistics for passes

In [ ]:
fle_kie_tags = pd.read_csv(r"handball_sample\MD05_Flensburg_Kiel\tags_flensburg_kiel_md5_s2324.csv", sep=";")

In [ ]:
fle_kie_tags["tag text"].unique()

In [ ]:
start_index = fle_kie_tags[fle_kie_tags["tag text"] == "game_start"].index[0]

In [ ]:
fle_kie_tags = fle_kie_tags.iloc[start_index:]
fle_kie_tags

In [ ]:
non_player_tags = ["ball_not_visible", "game_start", "time_check_position"]

player_tags = fle_kie_tags[~fle_kie_tags["tag text"].isin(non_player_tags)]

time_diffs = []
prev_time = None
for index, row in player_tags.iterrows():
    if row["tag text"] == "no_possession":
        prev_time = row["# time (in ms)"]
    elif prev_time is not None:
        time_diffs.append(row["# time (in ms)"] - prev_time)
        prev_time = None

time_diffs = [elem for elem in time_diffs if elem < 3000] # Remove every instance longer than three seconds 

average_time = statistics.mean(time_diffs)
std_time = statistics.stdev(time_diffs)

print(f"Average time from 'no_possession' to player's tag: {average_time} ms, standard deviation: {std_time} ms")

In [ ]:
# Histogram for time bins
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))
sns.histplot(time_diffs, bins=6, kde=False, palette="ch:s=.25,rot=-.25")
sns.set_theme(font_scale=1.3)
plt.title("Histogram of time durations for a pass in the game Flensburg vs Kiel")
plt.xlabel("Duration in ms")
plt.ylabel("Frequency")
# sns.despine(trim=True)
plt.show()

## Statistics for possession change

In [ ]:
match_training = pd.read_csv(r"handball_sample\match_training_model.csv", sep=";", index_col=0)

In [ ]:
ball_data = match_training[match_training["full name"].str.contains("ball")]
player_data = match_training[~match_training["full name"].str.contains("ball")]

aligned_data = player_data.merge(ball_data[["formatted local time", "x in m", "y in m", "z in m", "speed in m/s", "acceleration in m/s2", "metabolic power in W/kg", "acceleration load"]], 
                                on="formatted local time", how="left", suffixes=("_player", "_ball"))

aligned_data["distance_to_ball"] = np.sqrt(
    (aligned_data["x in m_player"] - aligned_data["x in m_ball"]) ** 2 +
    (aligned_data["y in m_player"] - aligned_data["y in m_ball"]) ** 2
)

In [ ]:
aligned_data

In [ ]:
cols_for_statistics = ["speed in m/s_player", "acceleration in m/s2_player", "metabolic power in W/kg_player", "acceleration load_player",
                       "speed in m/s_ball", "acceleration in m/s2_ball", "distance_to_ball"]

In [ ]:
# Timestamps where player has possession
possession_df = aligned_data[aligned_data["full name"] == aligned_data["tag text"]]
means_poss = round(possession_df[cols_for_statistics].mean(), 2)
stds_poss = round(possession_df[cols_for_statistics].std(), 2)

In [ ]:
# Timestamps with possession change
possession_copy = possession_df.copy()
possession_copy["prev_player"] = possession_copy["full name"].shift(1)

possession_change_df = possession_copy[possession_copy["full name"] != possession_copy["prev_player"]]
possession_change_df = possession_change_df.drop(columns=["full name"])

means_poss_change = round(possession_change_df[cols_for_statistics].mean(), 2)
stds_poss_change = round(possession_change_df[cols_for_statistics].std(), 2)

In [ ]:
# Other timestamps
no_possession_players = aligned_data[~aligned_data.index.isin(possession_df.index)]
means_no_poss = round(no_possession_players[cols_for_statistics].mean(), 2)
stds_no_poss = round(no_possession_players[cols_for_statistics].std(), 2)

summary_poss = means_poss.astype(str) + " (" + stds_poss.astype(str) + ")"
summary_poss_change = means_poss_change.astype(str) + " (" + stds_poss_change.astype(str) + ")"
summary_no_poss = means_no_poss.astype(str) + " (" + stds_no_poss.astype(str) + ")"

In [ ]:
statistics_table = pd.DataFrame({"Possession": summary_poss,
                                 "Possession change": summary_poss_change,
                                 "No possession": summary_no_poss})
statistics_table

## Examples for initial processing

In [ ]:
proc_ex = pd.read_csv(r"handball_sample\MD05_Flensburg_Kiel\tags_flensburg_kiel_md5_s2324.csv", sep=";").drop(["player"], axis=1)[:13]
proc_ex = proc_ex.drop(proc_ex[proc_ex["tag text"] == "no_possession"].index, axis=0).reset_index()
proc_ex = proc_ex.drop(["index"], axis=1)
start_time = proc_ex[proc_ex["tag text"] == "game_start"]["# time (in ms)"].iloc[0]
proc_ex

In [ ]:
proc_ex["# time (in ms)"] = proc_ex["# time (in ms)"] - start_time
proc_ex

In [ ]:
game_ex = pd.read_csv(r"handball_sample\MD05_Flensburg_Kiel\SG_Flensburg-Handewitt_vs._THW_phase_1.HZ_positions.csv", sep=";")[30:40]
game_ex

In [ ]:
kinexon_poss_start = pd.to_datetime(game_ex.loc[37]["formatted local time"])
proc_ex["adjusted_timestamp"] = proc_ex["# time (in ms)"].apply(lambda x: kinexon_poss_start + timedelta(milliseconds=x))
proc_ex

In [ ]:
def round_to_nearest_50ms(dt):
    """ Round the timestamps to nearest 50ms.
    
    Parameters:
    dt -- the datetime timestamps

    Returns:
    rounded_dt -- the timestamps rounded to nearest 50ms
    """
    # Convert microseconds to milliseconds and round to nearest 50 milliseconds
    milliseconds = round(dt.microsecond / 1000.0 / 50.0) * 50
    # Rebuild datetime with new milliseconds, adjusting for overflow
    new_second = dt.second
    if milliseconds >= 1000:
        milliseconds -= 1000
        new_second += 1
    # Ensure seconds don't overflow
    if new_second >= 60:
        new_second -= 60
        dt += timedelta(minutes=1)
    rounded_dt = dt.replace(second=new_second, microsecond=int(milliseconds * 1000))
    return rounded_dt

proc_ex["rounded_timestamp"] = proc_ex["adjusted_timestamp"].apply(round_to_nearest_50ms)
proc_ex

In [ ]:
# Forward fill tags df to have correct merging of match and tags df
video_start_time = proc_ex["rounded_timestamp"].iloc[0]
table_end_time = proc_ex["rounded_timestamp"].iloc[-1]

# Create timestamp df with frequency of 20Hz (frequency of Kinexon data) from game start to end time
all_timestamps = pd.date_range(start=video_start_time, end=table_end_time, freq="50L")
all_timestamps_df = pd.DataFrame(all_timestamps, columns=["timestamp"])

# Merge tags with all timestamps and forward fill
proc_ex_full = pd.merge(all_timestamps_df, proc_ex, left_on="timestamp", right_on="rounded_timestamp", how="left")
proc_ex_full["tag text"].replace(["game_start", "no_possession", "time_check_position"], np.nan, inplace=True)
proc_ex = proc_ex.columns.difference(["# time (in ms)", "adjusted_timestamp"])
for col in proc_ex:
    proc_ex_full[col] = proc_ex_full[col].ffill()
proc_ex_full.drop(["adjusted_timestamp", "rounded_timestamp", "# time (in ms)"], axis=1)